In [ ]:
import pandas as pd
import numpy as np
import matplotlib.plot as plt

In [ ]:
chunk_list = []
with pd.read_json('data/quotes-2020.json.bz2', lines=True, compression='bz2', chunksize=1_000_000) as df_reader:
    for chunk in df_reader:
        processed_chunk = clean_data(chunk)
        chunk_list.append(processed_chunk)
large_df = pd.concat(chunk_list, ignore_index=True)
large_df.to_csv(path_or_buf='data/clean-quotes-2020.bz2', compression='bz2')